##### Import data

In [1]:
from util_copy import load_fabric_data, extract_label_grouping, extract_label_grouping, load_fabric_images
import numpy as np
from matplotlib import image

In [2]:

path = r"C:/Users/Administrator/Desktop/PRML/Project/fabric_data/label_json/**/**.json"

fids, fdata = load_fabric_data(path)
ftype1, ftype2 = extract_label_grouping(fdata)


In [3]:
path = r"C:/Users/Administrator/Desktop/PRML/Project/fabric_data/temp/"
labels, imgs = load_fabric_images(path, fids, fdata, ftype2)


C:/Users/Administrator/Desktop/PRML/Project/fabric_data/temp/**/**.jpg


In [25]:
print(len(labels))
print(imgs[0])

3474
[[[115 102  49 121 104  32]
  [115 102  49 121 104  32]
  [124 107  57 127 105  42]
  ...
  [169 169 176 171 176 191]
  [170 169 177 169 175 191]
  [170 169 177 169 175 191]]

 [[115 102  49 121 104  32]
  [115 102  49 121 104  32]
  [124 107  57 127 105  42]
  ...
  [169 169 176 171 176 191]
  [170 169 177 169 175 191]
  [170 169 177 169 175 191]]

 [[116 104  50 119 103  32]
  [116 104  50 120 103  32]
  [124 109  58 127 105  43]
  ...
  [170 170 176 172 176 191]
  [171 170 177 171 175 190]
  [171 170 177 171 175 190]]

 ...

 [[ 96  24  27 138  47  52]
  [ 96  23  26 138  47  52]
  [ 95  20  21 138  46  51]
  ...
  [ 63  74  73  76  88  86]
  [ 63  75  74  75  88  86]
  [ 63  75  75  75  88  86]]

 [[ 96  24  27 140  48  53]
  [ 96  24  27 140  48  53]
  [ 95  20  22 140  47  52]
  ...
  [ 64  76  75  76  87  86]
  [ 65  77  77  76  88  86]
  [ 65  77  77  76  88  86]]

 [[ 96  24  27 140  48  53]
  [ 96  24  27 140  48  53]
  [ 95  20  22 140  47  53]
  ...
  [ 64  76  75  76 

In [4]:
n_samples = len(imgs)
print("Number of samples:", n_samples)

Number of samples: 3474


##### Padding
- https://blog.csdn.net/wuzqChom/article/details/74785643
- https://stackoverflow.com/questions/47697622/cnn-image-resizing-vs-padding-keeping-aspect-ratio-or-not/49882055#49882055
- https://stackoverflow.com/questions/43391205/add-padding-to-images-to-get-them-into-the-same-shape

In [5]:
import cv2

In [6]:
imgs = [cv2.resize(img,(200, 200)) for img in imgs]

3474


##### Split data

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
train_images, test_images, train_labels, test_labels = train_test_split(imgs, labels, test_size=0.3, random_state=1)

In [11]:
print(train_images.type)

AttributeError: 'list' object has no attribute 'type'

In [12]:
print("#Training data: {}\n#Testing data: {}\n#Class: {}".format(len(train_images), len(test_images), len(set(train_labels))))

#Training data: 2431
#Testing data: 1043
#Class: 9


##### Normalize data

In [13]:
train_images, test_images, train_labels, test_labels = np.array(train_images), np.array(test_images), np.array(train_labels), np.array(test_labels)
train_images, test_images = train_images / 255.0, test_images / 255.0

In [ ]:
train_images.shape

##### Generate fake data

##### Create model
- https://www.tensorflow.org/tutorials/images/classification

In [14]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [15]:
input_shape = (200, 200, 6)

In [16]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape = input_shape))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(9))

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 32)      1760      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 99, 99, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 97, 97, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 64)        36928     
_________________________________________________________________
flatten (Flatten)            (None, 135424)            0         
_________________________________________________________________
dense (Dense)                (None, 64)                8

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

##### Testing

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
print(test_acc)